# 02_peptide_level_analysis

## uptake plots

In [ ]:
# option 1

all_peps = [peptide for hdxms_data in hdxms_data_list for state in hdxms_data.states for peptide in state.peptides]
all_idfs = set([peptide.identifier for peptide in all_peps])

output_path = 'output'

if not os.path.exists(output_path):
    os.makedirs(output_path)
else:
    shutil.rmtree(output_path)
    os.makedirs(output_path)
    
    
for idf in all_idfs:
    uptake = UptakePlot(hdxms_data_list, idf)
    fig = uptake.uptakeplot
    fig.savefig(f"{output_path}/{idf}.png")

In [ ]:
# option 2, using the collection object

uptakes = UptakePlotsCollection(if_plot_fit=True)
uptakes.add_plot_all(hdxms_data_list)
# # save all the plots
# uptakes.save_plots('/tmp/test')

In option 2, the peptides that have high std across different experimental replicates will be automatically excluded

## pipetide comparison

`HDXStatePeptideCompares` can be created between two different states for comparison. It is a collection of all possible `HDXStatePeptideCompare`, where the same peptides are in both Proteinstate. HDXStatePeptideCompare
will find all the common timepoints and get the difference in D uptake. 

In [ ]:
state1 = 'APO'
state2 = 'RIB'

# first get the states
state1_list = [i.get_state(state1) for i in hdxms_data_list]
state2_list = [i.get_state(state2) for i in hdxms_data_list]

pep_compres = HDXStatePeptideCompares(state1_list, state2_list)
pep_compres.add_all_compare()

In [ ]:
# check the results
for i in pep_compres.peptide_compares[:10]:
    print('-----------------')
    print(i.compare_info)
    print(i.common_timepoints)
    print(i.deut_diff)
    

## pseudo residue compare

pseudo residue compare is a comparison of the average D uptake between two different `Proteinstate` of all the peptides that cover this residue. D here is the normalized values based on the full-D experiments (0-100)

In [2]:
res_compares = HDXStateResidueCompares([i for i in range(1, 320)], state1_list, state2_list)
res_compares.add_all_compare()

NameError: name 'HDXStateResidueCompares' is not defined

In [ ]:
# check the results
# deut_diff is the average deuteration difference of all peptides covering the residue
for i in res_compares.residue_compares[:5]:
    print('-----------------')
    print(i.compare_info)
    print(i.common_timepoints)
    print(i.deut_diff)
    

# Visualization on pymol structure

Both comparison objects can be projected to a pdb structure for Visualization

In [ ]:
# both of the compare objects can be exported as a pymol pse file for visualization
pdb_file = 'data/2dri.pdb'

create_compare_pymol_plot(pep_compres, colorbar_max=20, pdb_file=pdb_file, path=f'{output_path}')
create_compare_pymol_plot(res_compares, colorbar_max=20, pdb_file=pdb_file, path=f'{output_path}')

In [ ]:
#  it can be visualized as a heatmap
create_heatmap_compare_tp(pep_compres, 20)

In [ ]:
# make all the plots and save them

from itertools import product

items =[state.state_name for state in hdxms_data.states]
combinations = product(['APO'], [x for x in items if x != 'APO'])


OUTDIR = output_path
if not os.path.exists(OUTDIR):
    os.makedirs(OUTDIR)

for state1_name, state2_name in combinations:

    state1_list = [i.get_state(state1_name) for i in hdxms_data_list]
    state2_list = [i.get_state(state2_name) for i in hdxms_data_list]

    compare = HDXStatePeptideCompares(state1_list, state2_list)
    compare.add_all_compare()

    heatmap_compare_tp = create_heatmap_compare_tp(compare, 20)
    heatmap_compare_tp.savefig(f'{OUTDIR}/{state1}-{state2}-heatmap-tp.png')

    heatmap_compare = create_heatmap_compare(compare, 20)
    heatmap_compare.savefig(f'{OUTDIR}/{state1}-{state2}-heatmap.png')

    create_compare_pymol_plot(compare, colorbar_max=20, pdb_file=pdb_file, path=OUTDIR)



    res_compares = HDXStateResidueCompares([i for i in range(1, 320)], state1_list, state2_list)
    res_compares.add_all_compare()

    create_compare_pymol_plot(res_compares, 20, pdb_file=pdb_file, path=OUTDIR)